In [14]:
import boto3
import json
import time

def create_trust_policy():
    """Create the trust policy for Bedrock"""
    return {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "bedrock.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }

def create_embedding_policy(role_name, region):
    """Create policy for embedding model access"""
    return {
        "PolicyName": f"{role_name}-embedding",
        "PolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:InvokeModel"
                    ],
                    "Resource": [
                        f"arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1",
                        f"arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v2:0"
                    ]
                }
            ]
        }
    }

def create_kb_access_policy(role_name, region, account_id):
    """Create policy for knowledge base access"""
    return {
        "PolicyName": f"{role_name}-kb-access",
        "PolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:Retrieve",
                        "bedrock:RetrieveAndGenerate",
                        "bedrock:GetKnowledgeBase"
                    ],
                    "Resource": [
                        f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/*"
                    ]
                }
            ]
        }
    }

def create_custom_connector_policy(role_name):
    """Create policy for custom connector access"""
    return {
        "PolicyName": f"{role_name}-custom-connector",
        "PolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:GenerateQuery",
                        "sqlworkbench:GetSqlRecommendations"
                    ],
                    "Resource": "*"
                }
            ]
        }
    }

def create_oss_policy(role_name, region_name, account_number, collection_id):
       return {
           "PolicyName": f"{role_name}-oss",
           "PolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "aoss:APIAccessAll"
                    ],
                    "Resource": [
                        f"arn:aws:aoss:{region_name}:{account_number}:collection/{collection_id}"
                    ]
                }
            ]
        }
    }

def create_access_policy(role_name, vector_store_name, identity, role_arn):
       return {
           "PolicyName": f"{role_name}-access",
           "PolicyDocument": [
                        {
                            'Rules': [
                                {
                                    'Resource': ['collection/' + vector_store_name],
                                    'Permission': [
                                        'aoss:CreateCollectionItems',
                                        'aoss:DeleteCollectionItems',
                                        'aoss:UpdateCollectionItems',
                                        'aoss:DescribeCollectionItems'],
                                    'ResourceType': 'collection'
                                },
                                {
                                    'Resource': ['index/' + vector_store_name + '/*'],
                                    'Permission': [
                                        'aoss:CreateIndex',
                                        'aoss:DeleteIndex',
                                        'aoss:UpdateIndex',
                                        'aoss:DescribeIndex',
                                        'aoss:ReadDocument',
                                        'aoss:WriteDocument'],
                                    'ResourceType': 'index'
                                }],
                            'Principal': [identity, role_arn],
                            'Description': 'Easy data policy'}
                    ]
    }


def create_and_attach_policy(iam_client, role_name, policy_config):
    """Create and attach a policy to the role"""
    try:
        policy = iam_client.create_policy(
            PolicyName=policy_config["PolicyName"],
            PolicyDocument=json.dumps(policy_config["PolicyDocument"])
        )
        policy_arn = policy['Policy']['Arn']

        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Created and attached policy: {policy_config['PolicyName']}")
        return policy_arn
    except Exception as e:
        print(f"Error creating/attaching policy {policy_config['PolicyName']}: {str(e)}")
        raise

def create_bedrock_kb_role(role_name_prefix, role_name, iam_client, region, account_id):
    """Create the Bedrock knowledge base execution role with necessary policies"""

    created_policies = []

    try:
        # Create the IAM role
        print(f"Creating IAM role: {role_name}")
        role = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(create_trust_policy()),
            Description='Amazon Bedrock Knowledge Base Execution Role for custom connector'
        )

        # Allow time for role to propagate
        time.sleep(5)

        # Create and attach policies
        policies_to_create = [
            create_embedding_policy(role_name, region),
            create_kb_access_policy(role_name, region, account_id),
            create_custom_connector_policy(role_name),
        ]

        for policy_config in policies_to_create:
            policy_arn = create_and_attach_policy(iam_client, role_name, policy_config)
            created_policies.append(policy_arn)

        return {
            'role_name': role_name,
            'role_arn': role['Role']['Arn'],
            'policies': created_policies
        }

    except Exception as e:
        print(f"Error creating role: {str(e)}")
        cleanup_role_and_policies(role_name)
        raise

def cleanup_role_and_policies(role_name):
    """Clean up role and its policies"""
    iam_client = boto3.client('iam')
    
    try:
        # 1. List and detach managed policies
        managed_policies = iam_client.list_attached_role_policies(RoleName=role_name)
        for policy in managed_policies['AttachedPolicies']:
            iam_client.detach_role_policy(
                RoleName=role_name,
                PolicyArn=policy['PolicyArn']
            )
        
        # 2. List and delete inline policies
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        for policy_name in inline_policies['PolicyNames']:
            iam_client.delete_role_policy(
                RoleName=role_name,
                PolicyName=policy_name
            )
        
        # 3. Delete the role
        iam_client.delete_role(RoleName=role_name)
        print(f"Cleaned up role {role_name} and its policies")
        
    except Exception as e:
        print(f"Error during cleanup: {str(e)}")


In [22]:
print("========================================================================================")
print(f"Step 0 - Setup clients and variables")
print("========================================================================================")

session = boto3.session.Session()
role_name_prefix="bedrock-kb-custom-connector"
iam_client = session.client('iam')
sts_client = session.client('sts')
iam_client = session.client('iam')
aoss_client = session.client('opensearchserverless')
account_id = sts_client.get_caller_identity()['Account']
region = session.region_name
identity = sts_client.get_caller_identity()['Arn']

#self.region_name = session.region_name

Step 0 - Setup clients and variables


In [31]:
print("========================================================================================")
print(f"Step 1 - Creating Knowledge Base Execution Role () and Policies")
print("========================================================================================")

# Create the role and policies
role_name = f"{role_name_prefix}-{int(time.time())}"
role_info = create_bedrock_kb_role(role_name_prefix, role_name, iam_client, region, account_id)
print(f"Created role with ARN: {role_info['role_arn']}")
        
# Simulate using the role for your knowledge base operations
print("Role is ready for use with your knowledge base")


Step 1 - Creating Knowledge Base Execution Role () and Policies
Creating IAM role: bedrock-kb-custom-connector-1738799185
Created and attached policy: bedrock-kb-custom-connector-1738799185-embedding
Created and attached policy: bedrock-kb-custom-connector-1738799185-kb-access
Created and attached policy: bedrock-kb-custom-connector-1738799185-custom-connector
Created role with ARN: arn:aws:iam::044009490089:role/bedrock-kb-custom-connector-1738799185
Role is ready for use with your knowledge base


In [36]:
import json
import boto3
print(f"Step 2 - Creating OSS encryption, network and data access policies")
encryption_policy_name = 'bedrock-kb-encryption-policy'
def create_encryption_policy():
    # Create encryption policy
    try:
        encryption_policy = {
            "Rules": [
                {
                    "ResourceType": "collection",
                    "Resource": [
                        f"collection/bedrock-kb-*"
                    ]
                }
            ],
            "AWSOwnedKey": True
        }

        response = aoss_client.create_security_policy(
            name=encryption_policy_name,
            policy=json.dumps(encryption_policy),
            type='encryption'
        )
        return response

    except aoss_client.exceptions.ConflictException:
        print("Encryption policy already exists, retrieving existing policy...")
        existing_policy = aoss_client.get_security_policy(
            name=encryption_policy_name,
            type='encryption'
        )
        return existing_policy
    except Exception as e:
        print(f"Error creating/retrieving encryption policy: {str(e)}")
        raise



Step 2 - Creating OSS encryption, network and data access policies


In [39]:
def get_policy_arn(policy_name, iam_client):
    """Get the ARN of an IAM policy by its name"""
    
    try:
        # List policies with the given name
        response = iam_client.list_policies(
            Scope='Local',  # 'Local' for customer-managed policies, 'AWS' for AWS-managed
            PathPrefix='/',
            OnlyAttached=False
        )
        
        # Find the policy with matching name
        for policy in response['Policies']:
            if policy['PolicyName'] == policy_name:
                return policy['Arn']
                
        # If policy not found in first page, check pagination
        while response.get('IsTruncated', False):
            response = iam_client.list_policies(
                Scope='Local',
                PathPrefix='/',
                OnlyAttached=False,
                Marker=response['Marker']
            )
            
            for policy in response['Policies']:
                if policy['PolicyName'] == policy_name:
                    return policy['Arn']
        
        print(f"Policy '{policy_name}' not found")
        return None
        
    except Exception as e:
        print(f"Error getting policy ARN: {str(e)}")
        raise




In [40]:
# Example usage
try:
    policy_name = encryption_policy_name
    policy_arn = get_policy_arn(policy_name, iam_client)
    if policy_arn:
        print(f"Policy ARN: {policy_arn}")
except Exception as e:
    print(f"Error: {e}")

Error: get_policy_arn() missing 1 required positional argument: 'iam_client'


In [38]:
encryption_policy = create_encryption_policy()
print(encryption_policy)
get_policy_arn(encryption_policy_name)

Encryption policy already exists, retrieving existing policy...
{'securityPolicyDetail': {'createdDate': 1738779786559, 'lastModifiedDate': 1738779786559, 'name': 'bedrock-kb-encryption-policy', 'policy': {'Rules': [{'Resource': ['collection/bedrock-kb-*'], 'ResourceType': 'collection'}], 'AWSOwnedKey': True}, 'policyVersion': 'MTczODc3OTc4NjU1OV8x', 'type': 'encryption'}, 'ResponseMetadata': {'RequestId': '9ed86e96-2247-4075-83b0-2eefb1e1cd8a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9ed86e96-2247-4075-83b0-2eefb1e1cd8a', 'date': 'Wed, 05 Feb 2025 23:50:44 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '293', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
Policy 'bedrock-kb-encryption-policy' not found


In [18]:
cleanup_role_and_policies('bedrock-kb-custom-connector-1738797042')

Cleaned up role bedrock-kb-custom-connector-1738797042 and its policies


In [1]:

print("========================================================================================")
print(f"Step 3 - Creating OSS Collection (this step takes a couple of minutes to complete)")
print("========================================================================================")
print(f"Step 4 - Creating OSS Vector Index")
print("========================================================================================")
print(f"Step 5 - Creating Knowledge Base")
print("========================================================================================")

Step 2 - Creating Knowledge Base Execution Role () and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
Step 5 - Creating OSS Vector Index
Step 7 - Creating Knowledge Base
